In [230]:
import numpy as np
from scipy.spatial.distance import cosine as cosine_similarity
import random
import cProfile

# Binary search simulation

To understand the kind of problems that computers need to deal with on daily bases I prepared a little simulation.

You are given a hand of 20 sorted cards and your task is to find a particular card. Have fun!

In [172]:
def generated_ordered_cards():
    cards = []
    cards.extend(unichr(x) for x in range(127185, 127185 + 14))
    cards.extend(unichr(x) for x in range(127169, 127169 + 14))
    cards.extend(unichr(x) for x in range(127153, 127153 + 14))    
    cards.extend(unichr(x) for x in range(127137, 127137 + 14))
    return cards

In [175]:
DOMAIN = generated_ordered_cards()
NUM_SAMPLES = 20

class Problem(object):
    def __init__(self):
        self.elements = set()
        while len(self.elements) < NUM_SAMPLES:
            self.elements.add(random.choice(DOMAIN))
        self.elements = sorted(list(self.elements), key=lambda x: DOMAIN.index(x))
        self.hide_all()
        self.query = random.choice(self.elements)
    
    def ask(self, position):
        assert 0 <= position < NUM_SAMPLES
        self.visible[position] = True
        return self
    
    def hide_all(self):
        self.visible = [False for _ in range(NUM_SAMPLES)]
        
    def _repr_html_(self):
        els_html = []
        for el_idx in range(len(self.elements)):
            if self.visible[el_idx]:
                els_html.append("<td style='text-align:center'><font size='5'><b>%s</b></font></td>" % (self.elements[el_idx]))
            else:
                els_html.append("<td style='text-align:center;background:lightskyblue'<small><font color='grey'>%d</font></small></td>" % (el_idx,))
        header_html = u"<center><h1>Find %s! (♣ < ♦ < ♥ < ♠)</h1></center><br />" % (self.query,)
        table_html  = "<table width='100%%' height='50px' style='table-layout: fixed'><tr>%s</tr></table>" % ("".join(els_html))
        return header_html + table_html

In [180]:
p = Problem()
p

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19


In [181]:
p.ask(10)

0,1,2,3,4,5,6,7,8,9,🂶,11,12,13,14,15,16,17,18,19


In [182]:
p.ask(16)

0,1,2,3,4,5,6,7,8,9,🂶,11,12,13,14,15,🂢,17,18,19


# Binary search implemetation - GLOVE vectors

Glove vectors try to assign vectors of numbers to words in surprising ways. One interesting property they have is
ability to solve many analogies. For example:

$$
V_{\text{berlin}} - V_{\text{germany}} \approx V_{\text{paris}} - V_{\text{france}}
$$

Today we will try to write an algorithm that can quickly find vectors from Glove database

In [5]:
class Glove(object):
    def __init__(self, path):
        self.word_vector = []
        
        with open(path) as f:
            for line in f:
                if len(line) < 1:
                    break
                line = line.split(' ')
                word, vector = line[0], np.array([float(x) for x in line[1:]])
                self.word_vector.append((word, vector))
        self.word_vector.sort()
        
    def __call__(self, key):
        return self.find_vector(key)
        
    def find_vector(self, key):
        for word,vector in self.word_vector:
            if word == key:
                return vector
        raise KeyError(key)
        
    def find_closest_word(self, key_vector,blacklist=[]):
        best_similarity = float('inf')
        best_word = None
        for word, vector in self.word_vector:
            if word in blacklist:
                continue
            similarity = cosine_similarity(vector, key_vector)
            if best_similarity > similarity:
                best_similarity = similarity
                best_word = word
        return best_word

In [24]:
# Download from http://nlp.stanford.edu/projects/glove/
glove = Glove("/home/sidor/projects/Dali/data/glove/glove.6B.300d.txt")


In [52]:
# this call is quite slow...
def analogy(thisword, tothis, islikethis):
    key_vector = glove(tothis) - glove(thisword) + glove(islikethis)
    best_word = glove.find_closest_word(key_vector, blacklist=[thisword, tothis,islikethis])
    print("%s is to %s like %s to <%s>" % (thisword, tothis, islikethis, best_word,))

In [58]:
analogy("germany", "berlin", "france")

germany is to berlin like france to <paris>


In [46]:
analogy("movie", "movies", "school")

movie is to movies like school to <films>


In [57]:
analogy("movie", "actor", "school")

movie is to actor like school to <teacher>


In [55]:
analogy("smaller", "small", "bigger")

smaller is to small like bigger to <big>


In [56]:
analogy("smaller", "small", "big")

smaller is to small like big to <huge>


In [60]:
analogy("one", "two", "two")

one is to two like two to <three>


In [59]:
analogy("king", "queen", "man")

king is to queen like man to <woman>


### Finding a vector given a word
Notice that implementation of find_word above is quite naive:
    
```python
    def find_vector(self, key):
        for word,vector in self.word_vector:
            if word == key:
                return vector
        raise KeyError(key)
```

Every query to look up a word takes about 100ms. This is not very good if we want to run millions of such queries per second!

In [69]:
cProfile.run('glove("zebra")')

         4 function calls in 0.123 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.123    0.123 <ipython-input-5-dbe6e5a0c3c4>:14(__call__)
        1    0.123    0.123    0.123    0.123 <ipython-input-5-dbe6e5a0c3c4>:17(find_vector)
        1    0.000    0.000    0.123    0.123 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Implementation using binary search

Here we perform binary search algorithm. List of words `glove.word_vectors` is sorted alphabetically. Now we wish to find index `i`, such that `glove.word_vectors[i][0] == key` for some `key` word.

Let $l$ be the lowest index in our array (zero) and $h$ be the highest index in our array (in this case 400000 - 1). To find and index on which a particular word is stored we find the middle $m = (h + l) / 2$. If key at index $m$ (denoted $k_m$) is less than desired key $k^*$ we know that $k^*$ is on some index in range $(m, h)$ otherwise it is in range $(l, m-1)$. This idea can be recursively applied to all the subranges. Notice that with each query range size is halved, so the complexity of the solution is $O(\log{(h - l)})$

In [70]:
def find_vector(self, key, lo=None, hi=None, debug=False):
    # Make sure by default we search over entire table
    if lo is None:
        lo = 0
    if hi is None:
        hi = len(self.word_vector) - 1
    
    if lo > hi:
        raise KeyError(key)
    
    mid = (hi + lo) / 2
    word, vector = self.word_vector[mid]
    if debug:
        print("Looking for %s in range(%d, %d). Middle is %s" % (key, lo, hi, word))

    if word == key:
        return vector
    elif key < word:
        return self.find_vector(key, lo, mid - 1, debug=debug)
    else: # key > word
        return self.find_vector(key, mid + 1, hi, debug=debug)
    
Glove.find_vector = find_vector

In [71]:
vec = glove.find_vector("zebra", debug=True)

Looking for zebra in range(0, 399999). Middle is jurnal
Looking for zebra in range(200000, 399999). Middle is ramdass
Looking for zebra in range(300000, 399999). Middle is syme
Looking for zebra in range(350000, 399999). Middle is vadims
Looking for zebra in range(375000, 399999). Middle is wilbarger
Looking for zebra in range(387500, 399999). Middle is yevtushenko
Looking for zebra in range(393750, 399999). Middle is zeughaus
Looking for zebra in range(393750, 396873). Middle is zaa
Looking for zebra in range(395312, 396873). Middle is zaremba
Looking for zebra in range(396093, 396873). Middle is zehava
Looking for zebra in range(396093, 396482). Middle is zayu
Looking for zebra in range(396288, 396482). Middle is zeami
Looking for zebra in range(396386, 396482). Middle is zeder
Looking for zebra in range(396386, 396433). Middle is zebu
Looking for zebra in range(396386, 396408). Middle is zebic
Looking for zebra in range(396398, 396408). Middle is zebras
Looking for zebra in range(39

In [72]:
try:
    vec = glove.find_vector("ronrivest", debug=True)
except KeyError:
    print("Not found!")

Looking for ronrivest in range(0, 399999). Middle is jurnal
Looking for ronrivest in range(200000, 399999). Middle is ramdass
Looking for ronrivest in range(300000, 399999). Middle is syme
Looking for ronrivest in range(300000, 349998). Middle is sensationalism
Looking for ronrivest in range(300000, 324998). Middle is rs6
Looking for ronrivest in range(300000, 312498). Middle is rescue
Looking for ronrivest in range(306250, 312498). Middle is riveras
Looking for ronrivest in range(309375, 312498). Middle is romero
Looking for ronrivest in range(310937, 312498). Middle is rostekhnadzor
Looking for ronrivest in range(310937, 311716). Middle is rosaleda
Looking for ronrivest in range(310937, 311325). Middle is ronstadt
Looking for ronrivest in range(310937, 311130). Middle is ronayne
Looking for ronrivest in range(311034, 311130). Middle is rongbuk
Looking for ronrivest in range(311083, 311130). Middle is ronin
Looking for ronrivest in range(311107, 311130). Middle is ronni
Looking for ro

In [73]:
cProfile.run('glove("zebra")')

         21 function calls (5 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-5-dbe6e5a0c3c4>:14(__call__)
     17/1    0.000    0.000    0.000    0.000 <ipython-input-70-eec181511d7b>:1(find_vector)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Extra problems (not graded)
1. Here's another way of thinking about binary search - we are looking for higest $x$ such that property $p(x)$ is true. For example if we are looking for index of a key in array $p(x) = k_x \leq k^*$. Can you give examples of other properties that we can binary search over? What makes a property suitable for use in binary search?

2. Binary search can **not** be used to minimize a quadratic function. Can you find a similar algorithm that can?

In [ ]:
# Hint to problem 2
cyph = lambda x: chr((ord(x) + 64) % 128)
''.join(map(cyph, "\t.34%!$`/&`30,)44).'`2!.'%`).`47/`0)%#%3l`#/.3)$%2`30,)44).'`)4`).`4(2%%`0)%#%3n"))